<a href="https://colab.research.google.com/github/rvprieto/DataExpert_Assignment20/blob/main/Assignment20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bibliotecas

In [1]:
!pip install tpot

In [2]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt

from tpot import TPOTClassifier

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics.pairwise import cosine_similarity

# ***Tirar Warning***
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Arquivos Json

In [4]:
df_VideoSample = pd.read_json('/content/drive/MyDrive/DataExpert/Assignment20/Video_Games_sample.json', lines=True, orient='lines')

df_MetaVideo = pd.read_json('/content/drive/MyDrive/DataExpert/Assignment20/meta_Video_Games.json.gz', lines=True,  orient='lines')

#Explorando df_VideoSample

In [5]:
df_VideoSample.sample(5)

,overall,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,verified,vote,image
311336,5,"12 15, 2015",AE5YV3J4FWHT2,B00IAVDOS6,NaN,brooklyn boy,works great bought 2 of them,Five Stars,1450137600,1.0,NaN,NaN
77940,5,"05 31, 2011",A3FVD7M6QWPBP3,B00111SFEU,NaN,Rambo,I could not be happier with my new headset.. G...,PERFECT!!!,1306800000,1.0,NaN,NaN
154553,5,"12 3, 2012",A1PPTUIKPC6M5U,B0050SYX8W,{'Format:': ' Video Game'},Dylan J. Marquis,It's Halo...need I say more? There are so many...,The best one yet.,1354492800,NaN,NaN,NaN
284695,5,"03 29, 2014",A3CR798N9GRIP6,B00F27JGVA,NaN,Luis,this is the best choice so far on memory for t...,I recomended,1396051200,1.0,NaN,NaN
267355,4,"02 2, 2016",A3VAAGZKS0R5TD,B00E290JRE,{'Color:': ' Black'},Amazon Customer,Great for the price.,Four Stars,1454371200,1.0,NaN,NaN


In [6]:
df_VideoSample.shape

(453562, 12)

In [7]:
df_VideoSample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453562 entries, 0 to 453561
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         453562 non-null  int64  
 1   reviewTime      453562 non-null  object 
 2   reviewerID      453562 non-null  object 
 3   asin            453562 non-null  object 
 4   style           318160 non-null  object 
 5   reviewerName    453511 non-null  object 
 6   reviewText      453217 non-null  object 
 7   summary         453396 non-null  object 
 8   unixReviewTime  453562 non-null  int64  
 9   verified        384546 non-null  float64
 10  vote            57123 non-null   object 
 11  image           4651 non-null    object 
dtypes: float64(1), int64(2), object(9)
memory usage: 41.5+ MB


In [8]:
#Verificando % de NaN
pd.DataFrame(df_VideoSample.isna().sum(axis=0)).sort_values(by=0, ascending=False)/df_VideoSample.shape[0]

,0
image,0.989746
vote,0.874057
style,0.298530
verified,0.152164
reviewText,0.000761
summary,0.000366
reviewerName,0.000112
overall,0.000000
reviewTime,0.000000
reviewerID,0.000000


In [9]:
#Salvando dados que precisamos prever 
df_pred = df_VideoSample[df_VideoSample['verified'].isna()]

In [10]:
#Distribuição da variavel verified, observa-se que o dataset apresenta desbalanceamento
df_VideoSample['verified'].value_counts()

1.0    314250
0.0     70296
Name: verified, dtype: int64

In [11]:
df_VideoSample['vote'].value_counts()

2        17633
3         9003
4         5476
5         3953
6         2869
         ...  
206          1
1,239        1
610          1
299          1
377          1
Name: vote, Length: 391, dtype: int64

In [12]:
df_VideoSample.describe(include='all')

,overall,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,verified,vote,image
count,453562.000000,453562,453562,453562,318160,453511,453217,453396,4.535620e+05,384546.000000,57123,4651
unique,NaN,6066,361288,266,274,245576,376697,228253,NaN,NaN,391,4366
top,NaN,"03 8, 2013",A29BQ6B90Y1R5F,B00178630A,{'Format:': ' Video Game'},Amazon Customer,Good,Five Stars,NaN,NaN,2,[https://images-na.ssl-images-amazon.com/image...
freq,NaN,1661,34,7630,124632,35358,1421,91825,NaN,NaN,17633,3
mean,4.102689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.421743e+09,0.817197,NaN,NaN
std,1.394520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.133394e+07,0.386505,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.625824e+08,0.000000,NaN,NaN
25%,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.391904e+09,1.000000,NaN,NaN
50%,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.435277e+09,1.000000,NaN,NaN
75%,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.475107e+09,1.000000,NaN,NaN


In [13]:
#Vamos dropar as colunas Vote e image porque apresentam mais de + de 85% de valor NaN
#Vamos dropar a coluna Style pois o formato dict atrapalha a manipulação dos dados em DataFrame, como é apenas um exercicio vamos economizar tempo com isso
df_VideoSample.drop(columns=['vote','image','style'],inplace=True)

In [14]:
df_VideoSample.isna().sum()

overall               0
reviewTime            0
reviewerID            0
asin                  0
reviewerName         51
reviewText          345
summary             166
unixReviewTime        0
verified          69016
dtype: int64

In [15]:
df_VideoSample.duplicated().sum()

17525

In [16]:
#Dropar colunas duplicadas
df_VideoSample.drop_duplicates(inplace=True)

In [17]:
#Dropar todas as linhas que apresentem qualquer valor NaN, como o dataset é grande não vai ser um problema
#Obs: Limpar os valores NaN exceto na featura verified
df_VideoSample.dropna(how='any',subset=['reviewerName','reviewText','summary'],inplace=True)

In [18]:
df_VideoSample.isna().sum()

overall               0
reviewTime            0
reviewerID            0
asin                  0
reviewerName          0
reviewText            0
summary               0
unixReviewTime        0
verified          68353
dtype: int64

In [19]:
#Resetando o index (Vai facilitar juntar o y_pred após a modelagem)
df_VideoSample.reset_index(drop=True, inplace=True)

In [20]:
df_VideoSample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435503 entries, 0 to 435502
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         435503 non-null  int64  
 1   reviewTime      435503 non-null  object 
 2   reviewerID      435503 non-null  object 
 3   asin            435503 non-null  object 
 4   reviewerName    435503 non-null  object 
 5   reviewText      435503 non-null  object 
 6   summary         435503 non-null  object 
 7   unixReviewTime  435503 non-null  int64  
 8   verified        367150 non-null  float64
dtypes: float64(1), int64(2), object(6)
memory usage: 29.9+ MB


In [21]:
#Para conseguir rodar o modelo de classificação vamos precisar transformar as variaveis categorias, como o dataset  é com basicamente composto por variaveis do tipo categoricas vamos utilizar o OrdinalEncoder para evitar de criar um dataset com muitas colunas
#Isso ajudando o modelo a performar melhor
enc = OrdinalEncoder()
enc.fit(df_VideoSample)

df_encoder= enc.transform(df_VideoSample)
df_encoder = pd.DataFrame(df_encoder,columns = df_VideoSample.columns)

df_encoder

,overall,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,verified
0,0.0,201.0,17443.0,0.0,144487.0,67467.0,61555.0,5792.0,NaN
1,4.0,201.0,240791.0,0.0,94840.0,237478.0,150519.0,5792.0,1.0
2,4.0,5777.0,73248.0,0.0,9783.0,119993.0,92489.0,5780.0,1.0
3,4.0,5761.0,179746.0,0.0,148280.0,281978.0,176512.0,5779.0,0.0
4,4.0,5727.0,27622.0,0.0,188740.0,105341.0,118881.0,5778.0,1.0
...,...,...,...,...,...,...,...,...,...
435498,4.0,4438.0,260650.0,265.0,180039.0,336190.0,89768.0,6063.0,1.0
435499,0.0,4438.0,229707.0,265.0,86595.0,45599.0,31433.0,6063.0,1.0
435500,1.0,4405.0,155309.0,265.0,76113.0,201711.0,133706.0,6062.0,1.0
435501,0.0,4405.0,43752.0,265.0,184299.0,32268.0,121750.0,6062.0,NaN


In [22]:
df_encoder = df_encoder.set_index(df_VideoSample.index)

In [23]:
df_encoder.isna().sum()

overall               0
reviewTime            0
reviewerID            0
asin                  0
reviewerName          0
reviewText            0
summary               0
unixReviewTime        0
verified          68353
dtype: int64

In [24]:
#Split dos dados para treino e predição dos dados NaN
df_pred = df_encoder[df_encoder['verified'].isna()]

X_pred = df_pred.drop(columns='verified')


In [25]:
df_encoder_limpo = df_encoder.dropna(how='any')

df_encoder_limpo

,overall,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,verified
1,4.0,201.0,240791.0,0.0,94840.0,237478.0,150519.0,5792.0,1.0
2,4.0,5777.0,73248.0,0.0,9783.0,119993.0,92489.0,5780.0,1.0
3,4.0,5761.0,179746.0,0.0,148280.0,281978.0,176512.0,5779.0,0.0
4,4.0,5727.0,27622.0,0.0,188740.0,105341.0,118881.0,5778.0,1.0
5,4.0,6065.0,207387.0,0.0,99167.0,190847.0,56766.0,5768.0,1.0
...,...,...,...,...,...,...,...,...,...
435497,0.0,4438.0,253722.0,265.0,194098.0,272474.0,46844.0,6063.0,1.0
435498,4.0,4438.0,260650.0,265.0,180039.0,336190.0,89768.0,6063.0,1.0
435499,0.0,4438.0,229707.0,265.0,86595.0,45599.0,31433.0,6063.0,1.0
435500,1.0,4405.0,155309.0,265.0,76113.0,201711.0,133706.0,6062.0,1.0


In [26]:
#Conforme ja observado o dataset esta desbalanceado
df_encoder_limpo['verified'].value_counts()

1.0    303125
0.0     64025
Name: verified, dtype: int64

In [27]:
#Como o dataset apresenta grande volume de dados grande, vamos realizar um undersampling para equilibrar as classes e evitar de estourar a memoria do colab

X = df_encoder_limpo.drop(columns='verified')
y = df_encoder_limpo['verified']

under = RandomUnderSampler()

X_under, y_under = under.fit_resample(X, y) 

In [28]:
#Para ganharmos tempo com modelagem e hiper parametrização vamos utilizar o autoML TPOTClassifier

#Importando os pacotes
modelo = TPOTClassifier(generations = 2, population_size = 3, verbosity = 3)
#Essa biblioteca ja testa varios modelos e parametros para achar o mais otimizado com base nos dados de treino

In [29]:
#Ajustando os dados 
modelo.fit(X_under, y_under)

32 operators have been imported by TPOT.


Optimization Progress:   0%|          | 0/9 [00:00<?, ?pipeline/s]

Skipped pipeline #5 due to time out. Continuing to the next pipeline.

Generation 1 - Current Pareto front scores:

-1	0.4784303006638032	SGDClassifier(input_matrix, SGDClassifier__alpha=0.0, SGDClassifier__eta0=1.0, SGDClassifier__fit_intercept=True, SGDClassifier__l1_ratio=0.5, SGDClassifier__learning_rate=invscaling, SGDClassifier__loss=squared_hinge, SGDClassifier__penalty=elasticnet, SGDClassifier__power_t=1.0)

-2	0.48819211245607186	SGDClassifier(GaussianNB(input_matrix), SGDClassifier__alpha=0.0, SGDClassifier__eta0=1.0, SGDClassifier__fit_intercept=True, SGDClassifier__l1_ratio=0.5, SGDClassifier__learning_rate=invscaling, SGDClassifier__loss=squared_hinge, SGDClassifier__penalty=elasticnet, SGDClassifier__power_t=1.0)

Generation 2 - Current Pareto front scores:

-1	0.5015853182350645	SGDClassifier(input_matrix, SGDClassifier__alpha=0.0, SGDClassifier__eta0=1.0, SGDClassifier__fit_intercept=True, SGDClassifier__l1_ratio=0.5, SGDClassifier__learning_rate=invscaling, SGDClassif

TPOTClassifier(generations=2, population_size=3, verbosity=3)

In [30]:
#Criando o DataFrame com a predição

df_resultado = pd.DataFrame(modelo.predict(X_pred)).set_index(X_pred.index)

df_resultado

,0
0,0.0
12,0.0
13,0.0
19,0.0
23,0.0
...,...
435477,1.0
435485,1.0
435489,1.0
435493,0.0


In [31]:
df_resultado.value_counts()

1.0    60323
0.0     8030
dtype: int64

In [32]:
#Preenchendo os campos NaN
df_VideoSample['verified']= df_VideoSample['verified'].fillna(df_resultado[0],axis=0)

In [33]:
df_VideoSample.isna().sum()

overall           0
reviewTime        0
reviewerID        0
asin              0
reviewerName      0
reviewText        0
summary           0
unixReviewTime    0
verified          0
dtype: int64

In [34]:
df_VideoSample.shape

(435503, 9)

In [35]:
#Filtrando apenas as linhas verificadas, após o processo de classificação 
df_Video_recomend=df_VideoSample[df_VideoSample['verified']==1]


#Recomendação

In [36]:
#Dataset apresenta as informações de correlação entre os produtos
df_MetaVideo.sample()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
59875,"[Video Games, PC, Games, </span></span></span>]",,[],,Idols Galore! + Slave Pageant Bishoujo Aventur...,"[B07KFMLMZ2, B01M8I6U3X, B07CMKHYJ7, B00QMN7AX...",,by\n \n Animewild,[Officially Licensed],"[>#83,230 in Video Games (See Top 100 in Video...","[B00QMO21N2, B00N5A9VLQ, B071ZZWKCS, B0026M4DI...",Video Games,,,\n\t\t ...,B00D9E3312,[],[],{}


In [37]:
#Vamos juntar tudo em um dataset
df_final= df_Video_recomend.merge(df_MetaVideo, how="left", on='asin')

df_final.sample()

,overall,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,verified,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,imageURL,imageURLHighRes,details
295821,5,"08 25, 2015",A393P67PW91PJD,B00N4ABT1C,ash,A+,Five Stars,1440460800,1.0,"[Video Games, Nintendo 3DS & 2DS, Interactive ...",,[Yoshi amiibo],,Yoshi amiibo (Super Smash Bros Series),"[B00N4ABMG4, B00O9GP99S, B0158VW8NM, B00Z1DX1B...",,by\n \n Nintendo,[Team up with your amiibo or play against frie...,"[>#7,557 in Video Games (See Top 100 in Video ...","[B00S7O6R62, B00O9GPAVU, B00N4ABMG4, B00O92ONB...",Video Games,,,\n\t\t ...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}


In [38]:
#Aproximada 80% dos produtos tem avaliação geral entre 4 e 5
df_final['overall'].value_counts()/df_final.shape[0]

5    0.663727
4    0.141545
1    0.080047
3    0.068019
2    0.046663
Name: overall, dtype: float64

In [39]:
#Para o nosso modelo de recomendação vamos fazer a recomendação pelos top200 produtos mais bem avaliados
product_rated=df_final.groupby(by='asin',as_index=False).agg({'overall':np.mean}).rename(columns={'overall':'overall_rate'})
shop_count=product_rated.set_index('asin')

top200_avg_products_score=shop_count.sort_values('overall_rate',ascending=False).head(200) 

top200_avg_products_score=top200_avg_products_score.reset_index()

top200_avg_products_score

,asin,overall_rate
0,B00AKIPBNS,4.810624
1,B00DC7G0GG,4.800771
2,B00UG63VDG,4.800589
3,B0017KIBAI,4.792510
4,B00K848IH0,4.775845
...,...,...
195,B00BGA9YZK,4.061111
196,B000NUBY0C,4.043771
197,B018K6KV68,4.040870
198,B00BDS415I,4.037456


In [40]:
#Para o caso do produto pesquisado não se encontrar na matriz de similaridade podemos deixar uma lista dos 10 mais vendidos pronta para recomendar

product_count=df_final.groupby(by='asin',as_index=False).agg({'verified':"count"}).rename(columns={'verified':'count'})
shop_count=product_count.set_index('asin')

top10_products=shop_count.sort_values('count',ascending=False).head(10)
top10_products=top10_products.reset_index()

top10_products

,asin,count
0,B00HTK1NCS,5875
1,B004RMK57U,4863
2,B00KKAQYXM,3946
3,B003ZSP0WW,3758
4,B00JJNQG98,3685
5,B00009OY9U,3454
6,B00E4MQODC,3202
7,B000B9RI14,3151
8,B00DBDPOZ4,2850
9,B000HZFCT2,2795


In [41]:
#Para nossa matriz de similaridade vamos considerar a similaridade dos top200 produtos entre as marcas(brand)
matriz_similaridade = df_final[df_final["asin"].isin(top200_avg_products_score["asin"])] \
  .pivot_table(index="asin", columns="brand", values="overall").fillna(0)

matriz_similaridade.shape

(200, 62)

In [42]:
#Criando a matriz de similaridade
from sklearn.metrics.pairwise import cosine_similarity
cos_item = cosine_similarity(matriz_similaridade)

print(cos_item)
cos_item.shape

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


(200, 200)

In [43]:
#Função para recomendação
def recomendar(asin):
  print(f"Produto observado: {asin}.\n")
  try:
    asin_select = matriz_similaridade.index.tolist().index(asin)
    top10_recomendados = np.argsort(-cos_item[asin_select])[1:11]
    print(f"Retornando os 10 produtos mais similares de: {asin}.")
    return matriz_similaridade.index[top10_recomendados].tolist()
  except ValueError as e:
    print(f"{asin}, não está incluso na matriz de recomendação. Retornando os Top10 comprados")
    return top10_products['asin'].tolist()

#Testando o modelo de recomendação

In [44]:
asin = "B00DC7G0GG"
produtos_similares = recomendar(asin)

produtos_similares

Produto observado: B00DC7G0GG.

Retornando os 10 produtos mais similares de: B00DC7G0GG.


['B0013E9HP6',
 'B0017KIBAI',
 'B00S1LRX3W',
 'B002BRZ9G0',
 'B002BSA3EM',
 'B00DC7G2W8',
 'B00MUY0OFU',
 'B00LCHZRIK',
 'B00L59D9HG',
 'B0053B5RGI']

In [45]:
asin = "B07FFCN3DJ"
produtos_similares = recomendar(asin)

produtos_similares

Produto observado: B07FFCN3DJ.

B07FFCN3DJ, não está incluso na matriz de recomendação. Retornando os Top10 comprados


['B00HTK1NCS',
 'B004RMK57U',
 'B00KKAQYXM',
 'B003ZSP0WW',
 'B00JJNQG98',
 'B00009OY9U',
 'B00E4MQODC',
 'B000B9RI14',
 'B00DBDPOZ4',
 'B000HZFCT2']

In [46]:
asin = "B01FKPC7UA"
produtos_similares = recomendar(asin)

produtos_similares

Produto observado: B01FKPC7UA.

B01FKPC7UA, não está incluso na matriz de recomendação. Retornando os Top10 comprados


['B00HTK1NCS',
 'B004RMK57U',
 'B00KKAQYXM',
 'B003ZSP0WW',
 'B00JJNQG98',
 'B00009OY9U',
 'B00E4MQODC',
 'B000B9RI14',
 'B00DBDPOZ4',
 'B000HZFCT2']

In [165]:
asin = "produto_novo"
produtos_similares = recomendar(asin)

produtos_similares

Produto observado: produto_novo.

produto_novo, não está incluso na matriz de recomendação. Retornando os 10 comprados


['B00HTK1NCS',
 'B004RMK57U',
 'B00KKAQYXM',
 'B003ZSP0WW',
 'B00JJNQG98',
 'B00009OY9U',
 'B00E4MQODC',
 'B000B9RI14',
 'B00DBDPOZ4',
 'B000HZFCT2']